In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import sqlite3

In [15]:
huddf = pd.read_csv('data/hud_insured_multifamily.csv')

C:\Users\mdalt\AppData\Local\Temp\ipykernel_2296\3926192688.py:1: DtypeWarning: Columns (2,19,22,35,36,41,89,90,91,92,95,103,109,111,121,123,125,126,127,164,169,170,176,205,209,217,274) have mixed types. Specify dtype option on import or set low_memory=False.
  huddf = pd.read_csv('data/hud_insured_multifamily.csv')


In [3]:
huddf.head

<bound method NDFrame.head of        OBJECTID  PROPERTY_ID      UGLG_KEY HUB_NAME_TEXT  \
0           1.0  800225589.0       3457600           OHP   
1           2.0  800225933.0       4878544  Headquarters   
2           3.0  800225983.0       1711202           OHP   
3           4.0  800226573.0       5327855           OHP   
4           5.0  800226611.0       1379388           OHP   
...         ...          ...           ...           ...   
16519   16519.0  800254374.0  3401343800.0      New York   
16520   16520.0  800255132.0     4829000.0    Fort Worth   
16521   16521.0  800255134.0     1963525.0   Kansas City   
16522   16522.0  800255144.0       24035.0     Baltimore   
16523   16523.0  800255510.0     2674960.0       Detroit   

      SERVICING_SITE_NAME_TEXT PROJECT_MANAGER_NAME_TEXT  \
0                          OHP                       NaN   
1          Headquarters - BRSC          LESLEY N JOHNSON   
2                          OHP                       NaN   
3        

In [4]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


## Clean Data


In [6]:
raw_df = pd.read_csv('data/hud_insured_multifamily.csv')


C:\Users\mdalt\AppData\Local\Temp\ipykernel_2296\3991870264.py:1: DtypeWarning: Columns (2,19,22,35,36,41,89,90,91,92,95,103,109,111,121,123,125,126,127,164,169,170,176,205,209,217,274) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv('data/hud_insured_multifamily.csv')


In [7]:
raw_df.head

<bound method NDFrame.head of        OBJECTID  PROPERTY_ID      UGLG_KEY HUB_NAME_TEXT  \
0           1.0  800225589.0       3457600           OHP   
1           2.0  800225933.0       4878544  Headquarters   
2           3.0  800225983.0       1711202           OHP   
3           4.0  800226573.0       5327855           OHP   
4           5.0  800226611.0       1379388           OHP   
...         ...          ...           ...           ...   
16519   16519.0  800254374.0  3401343800.0      New York   
16520   16520.0  800255132.0     4829000.0    Fort Worth   
16521   16521.0  800255134.0     1963525.0   Kansas City   
16522   16522.0  800255144.0       24035.0     Baltimore   
16523   16523.0  800255510.0     2674960.0       Detroit   

      SERVICING_SITE_NAME_TEXT PROJECT_MANAGER_NAME_TEXT  \
0                          OHP                       NaN   
1          Headquarters - BRSC          LESLEY N JOHNSON   
2                          OHP                       NaN   
3        

In [8]:
raw_df = raw_df.drop(columns=["PROPERTY_ID", "UGLG_KEY", "AUTOMATIC_GEOCODE_IND"])

In [9]:
raw_df = raw_df.dropna()

In [10]:
raw_df = raw_df.fillna(0)


In [11]:
raw_df.to_csv(('data/hud_insured_multifamily_clean.csv'), index=False)

## SQL + ANSWERS

In [18]:
## connect to a db
conn = sqlite3.connect('database.db')

create_table_command = """
CREATE TABLE IF NOT EXISTS HUD_Insured_Multifamily_Properties (
    ID INTEGER PRIMARY KEY,
    ROOMS INTEGER,
    BATHROOMS INTEGER,
    UNITS INTEGER,
    STORIES INTEGER
)
"""

# Function to execute the create table command
def execute_create_table(conn):
    conn.execute(create_table_command)
    conn.commit()

# Call the function to create the table
execute_create_table(conn)


In [19]:
### insert values into the table

insert_values = """
INSERT INTO HUD_Insured_Multifamily_Properties (ID, ROOMS, BATHROOMS, UNITS, STORIES)
VALUES (?, ?, ?, ?, ?)

SyntaxError: incomplete input (3683024685.py, line 3)

In [20]:
### function to insert values based on the query above

def insert_values(conn, values):
    conn.execute(insert_values, values)
    conn.commit()


In [21]:
### execute insert values

insert_values(conn, pd.read_csv('input_data/clean/HUD_Insured_Multifamily_Properties_Cleaned.csv')) 

# query the table

table = pd.read_sql_query("SELECT * FROM HUD_Insured_Multifamily_Properties", conn)


FileNotFoundError: [Errno 2] No such file or directory: 'input_data/clean/HUD_Insured_Multifamily_Properties_Cleaned.csv'

In [22]:
### answer the questions
questions = [
    "What is the average number of rooms per property?",
    "What is the average number of bathrooms per property?",
    "What is the average number of units per property?",
    "What is the average number of stories per property?",
    "What is the average number of units per property?",
]

In [23]:
#blank array with answers to the questions ^
answers = []

# "What is the average number of rooms per property?",
query_1 = """
SELECT AVG(ROOMS) FROM HUD_Insured_Multifamily_Properties
"""

In [24]:
answers.append(conn.execute(query_1).fetchone()[0])

In [25]:
### what is the average number of bathrooms per property?
query_2 = """
SELECT AVG(BATHROOMS) FROM HUD_Insured_Multifamily_Properties
"""

answers.append(conn.execute(query_2).fetchone()[0])


In [26]:
### create dataframe from questions and answers

querys_and_answers = pd.DataFrame({
    "Questions": questions,
    "Answers": answers
})

querys_and_answers.to_csv('output/Questions_and_Answers.csv', index=False)


ValueError: All arrays must be of the same length